
# Serving Trained model with basic BentoMl code

In [1]:
!git clone https://github.com/grayfactory/Toxicity-Under-Context-Detect.git
%cd Toxicity-Under-Context-Detect/
!ls

Cloning into 'Toxicity-Under-Context-Detect'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 31 (delta 10), reused 28 (delta 7), pack-reused 0
Unpacking objects: 100% (31/31), done.
Checking out files: 100% (14/14), done.
/content/Toxicity-Under-Context-Detect
data	   Preprocessing_data_and_prepare4training.ipynb  utils
model_ckp  README.md


In [ ]:
!pip install transformers
!pip install datasets
!pip install -q bentoml

In [ ]:
# for load model
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [3]:
%%writefile toxic_sentence_detection.py

import torch
from transformers import ElectraTokenizer, ElectraForSequenceClassification
import torch.nn.functional as F

import bentoml
from bentoml.frameworks.transformers import TransformersModelArtifact
from bentoml.adapters import DataframeInput, JsonInput


labels = ['Non-Toxic','Toxic']

@bentoml.env(infer_pip_packages=True)
@bentoml.artifacts([TransformersModelArtifact('ElectraModel')])
class ToxicClassifier(bentoml.BentoService):

  # @bentoml.utils.cached_property
  def tokenize(self, parsed_json, max_len=256):

    tokenizer = self.artifacts.ElectraModel.get("tokenizer")
    # print(parsed_json)
    # 주의
    # post로 요청될때는 list 속에 넣어서 전달됨
    if isinstance(parsed_json, list):
      src_text = parsed_json[0].get("text")
      src_parent = parsed_json[0].get("parent")
    elif isinstance(parsed_json, dict):
      src_text = parsed_json.get("text")
      src_parent = parsed_json.get("parent")

    tokens_a = tokenizer.tokenize(src_parent) # parent context first
    tokens_b = tokenizer.tokenize(src_text)

    # simple huristic
    while True:
      total_length = len(tokens_a) + len(tokens_b)
      if total_length <= max_len:
        break
      if len(tokens_a) > len(tokens_b):
        tokens_a.pop()
      else:
        tokens_b.pop()

    tokens = [tokenizer.cls_token, *tokens_a, tokenizer.sep_token]
    segment_ids = [0]*len(tokens)
    if tokens_b:
      tokens += [*tokens_b, tokenizer.sep_token]
      segment_ids += [1]*(len(tokens_b)+1)

    # mask has 1 for real tokens and - for padding tokens
    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    attention_mask = [1] * len(tokens)

    while len(input_ids) < max_len:
      input_ids.append(0)
      attention_mask.append(0)
      segment_ids.append(0)

    encoding = dict()

    encoding['input_ids'] = input_ids
    encoding['attention_mask'] = attention_mask
    encoding['segment_ids'] = segment_ids

    # return encoding
    return {
      'input_ids' : torch.tensor(encoding['input_ids'], dtype=torch.long).unsqueeze(0), 
      'attention_mask' : torch.tensor(encoding['attention_mask'], dtype=torch.float32).unsqueeze(0),
      'token_type_ids' : torch.tensor(encoding['segment_ids'], dtype=torch.long).unsqueeze(0),
    }


  @bentoml.api(input=JsonInput(), batch=True)
  def predict(self, parsed_json):
    
    encoding = self.tokenize(parsed_json=parsed_json)
    
    model = self.artifacts.ElectraModel.get("model")
    y_pred = model(input_ids = encoding['input_ids'],
                  attention_mask = encoding['attention_mask'],
                  token_type_ids = encoding['token_type_ids'])[0]
    prob, pred = torch.max(F.softmax(y_pred, dim=1), dim=1)

    # [class label, index, predict probability]
    return [labels[pred.item()], pred.item(), prob.item()]





Writing toxic_sentence_detection.py


In [4]:
import torch
from transformers import ElectraTokenizer, ElectraForSequenceClassification
from pathlib import Path

# set fine-tuning model ckp path
model_ckp = Path('/content/gdrive/MyDrive/toxic_nlp/Electra-model-save/electra-all-merged.pt')
if model_ckp.exists():
  pass
else :
  model_ckp = Path('model_ckp/electra-generated-data-full-merged.pt')

model_to_use = 'google/electra-small-discriminator'

# model & tokenizer load from transformers
model = ElectraForSequenceClassification.from_pretrained(model_to_use)
model.load_state_dict(torch.load(model_ckp, map_location=torch.device('cpu')))
tokenizer = ElectraTokenizer.from_pretrained(model_to_use)

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier

In [5]:
from toxic_sentence_detection import ToxicClassifier

bento_svc = ToxicClassifier()
artifact = {"model":model, "tokenizer": tokenizer}
bento_svc.pack("ElectraModel", artifact)


[2021-07-10 05:42:37,684] WARNING - pip package requirement `bentoml==0.13.0` not found in current python environment


In [6]:
test_input = {"parent": "I will kill myself right now", 
              "text":"That is the best news ever"}
bento_svc.predict(test_input)


['Toxic', 1, 0.7257377505302429]

In [ ]:
saved_path = bento_svc.save()
print(saved_path)

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


[2021-07-09 06:16:05,390] INFO - BentoService bundle 'ToxicClassifier:20210709061604_6B5F95' saved to: /root/bentoml/repository/ToxicClassifier/20210709061604_6B5F95
/root/bentoml/repository/ToxicClassifier/20210709061604_6B5F95


In [ ]:
import bentoml

# Test loaded bentoml service:
bento_toxic = bentoml.load(saved_path)

test_input = [{"parent": "Which baking dish is best to bake a banana bread ?","text":"What? fuck off"}]

# return class label & label index & probability!
bento_toxic.predict(test_input)

[2021-07-09 06:16:13,176] WARNING - Module `toxic_sentence_detection` already loaded, using existing imported module.
[2021-07-09 06:16:13,403] WARNING - pip package requirement xgboost already exist


['Toxic', 1, 0.6912020444869995]

In [ ]:
!bentoml serve ToxicClassifier:latest --run-with-ngrok

In [ ]:
curl -X POST http://localhost:5000/predict -H 'Content-Type: application/json' -d '{"text": "Which baking dish is best to bake a banana bread ?","parent":"What? fuck off"}'

In [ ]:
test_input = [
              {"parent": "I will kill myself right now", 
              "text":"That is the best news ever"},
              {"parent": "Which baking dish is best to bake a banana bread ?",
               "text":"What? fuck off"}]

In [ ]:
!bentoml run ToxicClassifier:latest predict --input '{"parent": "I will kill myself right now", "text":"That is the best news ever"}, {"parent": "Which baking dish is best to bake a banana bread ?","text":"What? fuck off"}] '

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
[2021-07-09 06:21:40,685] INFO - Getting latest version ToxicClassifier:20210709061604_6B5F95
[2021-07-09 06:21:41,816] ERROR - Error caught in API function:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/bentoml/service/inference_api.py", line 176, in wrapped_func
    return self._user_func(*args, **kwargs)
  File "/root/bentoml/repository/ToxicClassifier/20210709061604_6B5F95/ToxicClassifier/toxic_sentence_detection.py", line 76, in predict
    encoding = self.tokenize(parsed_json=parsed_json)
  File "/root/bentoml/repository/ToxicClassifier/20210709061604_6B5F95/ToxicClassifier/toxic_sentence_detection.py", line 25, in tokenize
  

In [ ]:
!bentoml run ToxicClassifier:latest predict --input '{"parent": "I will kill myself right now", "text":"That is the best news ever"}'

/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
[2021-07-09 06:24:33,259] INFO - Getting latest version ToxicClassifier:20210709061604_6B5F95
2021-07-09 06:24:34.954825: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
[2021-07-09 06:24:36,503] INFO - {'service_name': 'ToxicClassifier', 'service_version': '20210709061604_6B5F95', 'api': 'predict', 'task': {'data': '{"parent": "I will kill myself right now", "text":"That is the best news ever"}', 'task_id': '4d61f285-ed00-46da-ac1a-1d5bc2870a77', 'cli_args': ('--input', '{"parent": "I will kill myself right now", "text":"That is the best news ever"}'), 'inference_job_args': {}}, 'result': {'data': 